**Trabajo Final**

> Profesor: Luis Cajachahua

> Curso: MLE

> Alumno: Miguel Alberto LUIS CORDOVA

> Email: Miguel.Luis@outlook.com

> Caso: Póliza de Seguros (Inferencia de Ingresos)

## Instalamos librerías 

In [1]:
# Import necessary libs
import pandas as pd
import numpy as np
from math import sqrt

import pickle

import warnings
warnings.filterwarnings('ignore')
from scipy import stats

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

## Modelos de regresion
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm           
import statsmodels.formula.api as smf  

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Métricas de evaluación
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score

# Separar train and test
from sklearn.model_selection import train_test_split

# Estadística y matemáticas
import scipy.stats as scy
from scipy.stats import kurtosis

# Crea una semilla cualesquiera
seed = 16
np.random.seed(seed)

In [2]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 30

# Script 1: Preparación de datos para el entrenamiento


In [3]:
#Ruta del Archivo
mainpath = "../data"
filename = f'{mainpath}/data_Inferencia_Ingresos.xlsx'
dataset = pd.read_excel(filename)
dataset_original = pd.read_excel(filename)

In [4]:
#Aplicar función de logaritmo al Ingreso
dataset['ingres'] = np.log(dataset['ingres'])

In [5]:
#Creación de función para eliminar valores atípicos (considerando el IQR)
def Remove_Outlier(df_in, col_name):
  q1 = df_in[col_name].quantile(0.25)
  q3 = df_in[col_name].quantile(0.75)
  iqr = q3-q1 #Interquartile range
  rango_inicio = q1-1.5*iqr
  rango_fin = q3+1.5*iqr
  #print('Columna: ',col_name)
  #print('  IQR - Rango Inicio: ',rango_inicio)
  #print('  IQR - Rango Fin: ',rango_fin)
  df_out = df_in.loc[(df_in[col_name] > rango_inicio) & (df_in[col_name] < rango_fin)]

  registros_eliminados = df_in.shape[0] - df_out.shape[0]
  
  #print('Registros')
  #print('  Inicial:    ', '{:,}'.format(df_in.shape[0]) ,"-")
  #print('  Eliminados:   ','{:,}'.format(registros_eliminados))
  #print('  ----------------------')
  #print('  Final:      ', '{:,}'.format(df_out.shape[0]))

  return df_out

## Eliminación de valores atípicos

In [6]:
dataset_rmv_01 = Remove_Outlier(dataset, 'AniosDireccion')
dataset_rmv_02 = Remove_Outlier(dataset_rmv_01, 'Aniosempleo')
dataset_rmv_03 = Remove_Outlier(dataset_rmv_02, 'Aniosresiden')
dataset = dataset_rmv_03


In [7]:
# Mantener sólo las variables relevantes para el Modelo
dfp = dataset[['edad','AniosDireccion','Gastocoche','Aniosempleo','Aniosresiden','ingres']]
dfp.to_csv("../data/raw/model_train.csv")

# Script 2: Código de Entrenamiento

In [8]:
# Cargar la tabla transformada
df = pd.read_csv("../data/raw/model_train.csv")
X_train = df.drop(['ingres'],axis=1)
y_train = df[['ingres']]

### SKlearn (Linear Regression)

In [9]:
# Entrenamos el modelo con toda la muestra
lm = LinearRegression()
y_sqrt = np.sqrt(y_train)
y_log = np.log(y_train)
lm.fit(X_train,y_train)

LinearRegression()

In [10]:
# Guardamos el modelo entrenado para usarlo en produccion
filename = '../models/best_model.pkl'
pickle.dump(lm, open(filename, 'wb'))

# Script 3: Preparación de Datos de Validación

In [11]:
# Leemos la tabla de validación
df = pd.read_csv("../data/raw/model_validation.csv")

In [12]:
#Aplicar función de logaritmo al Ingreso
df['ingres'] = np.log(dataset['ingres'])

In [13]:
# Mantener sólo las variables relevantes para el Modelo
dfp = dataset[['edad','AniosDireccion','Gastocoche','Aniosempleo','Aniosresiden','ingres']]
dfp.to_csv("../data/raw/model_val.csv")

# Script 4: Código de Validación

In [14]:
# Cargar la tabla transformada
df = pd.read_csv("../data/raw/model_val.csv")
X_test = df.drop(['ingres'],axis=1)
y_test = df[['ingres']]

In [15]:
# Leemos el modelo entrenado!
filename = '../models/best_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [16]:
# Predecimos sobre el set de datos de implementacion con el modelo entrenado
y_pred_test=model.predict(df.drop(['ingres'],axis=1)) 

In [17]:
## Metricas de validación
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calc_metrics(y_test,predictions_lr):
    RMSE_LR = np.sqrt(metrics.mean_squared_error(y_test, predictions_lr))
    print('MAE:', metrics.mean_absolute_error(y_test, predictions_lr))
    print('MSE:', metrics.mean_squared_error(y_test, predictions_lr))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions_lr)))
    print('MAPE:', mean_absolute_percentage_error(y_test, predictions_lr))

In [18]:
#Creación de función para realizar gráfico de Predicción
def Grafico_Prediccion(title, y_test_in, predictions_in):
 # Valores predecidos vs valores reales (Linealidad)
  y_pred = predictions_in.copy()
  plt.scatter(y_test_in, y_pred, edgecolors=(0, 0, 0), alpha = 0.4)
  plt.plot([y_test_in.min(), y_test_in.max()], 
          [y_test_in.min(), y_test_in.max()],'k--', color = 'black', lw=2)
  plt.title('(' + title + ') Valor predicho vs valor real', fontsize = 20, fontweight = "bold")
  plt.xlabel('Real')
  plt.ylabel('Predicción')

In [19]:
def save_plot(title):
    plt.title(title)
    fig = plt.gcf()
    filename = title.replace(" ", "_").lower()
    fig.savefig('../' + filename, dpi=500)
    plt.clf()

In [20]:
Grafico_Prediccion('Regresión Lineal', y_test, y_pred_test.copy())
save_plot('Gráfico Dispersión')

<Figure size 432x288 with 0 Axes>

# Script 5: Preparación de Datos de Score (Automatización)

In [21]:
# Leemos la tabla de entrenamiento
df = pd.read_csv("../data/raw/model_train.csv")

In [22]:
#Aplicar función de logaritmo al Ingreso
dataset['ingres'] = np.log(dataset['ingres'])

In [23]:
# Mantener sólo las variables relevantes para el Modelo
dfp = dataset[['edad','AniosDireccion','Gastocoche','Aniosempleo','Aniosresiden','ingres']]
dfp.to_csv("../data/processed/model_score.csv")

# Script 6: Código de Scoring (Automatización)

In [24]:
# Cargar la tabla transformada
df = pd.read_csv("../data/processed/model_score.csv").set_index('Unnamed: 0')

In [25]:
# Leemos el modelo entrenado!
filename = '../models/best_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [26]:
# Predecimos sobre el set de datos de implementacion con el modelo entrenado
scores=model.predict(df)

In [27]:
# Exportamos el resultado del modelo para cargarlo en el Feature Store o Data Mart de Modelos
# Le asignamos nombres a las columnas
df_score = pd.DataFrame(scores, columns=['ingres'])
# Exportamos la solucion
df_score.to_csv('../data/scores/final_score.csv')